In [1]:
# 导入必要的库
import qlib
from qlib.data import D
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 添加项目路径
sys.path.append('../qlib_workflow')
from indicators.price_startup_indicator import PriceStartupIndicator, create_startup_fields


In [2]:
# 初始化qlib并创建指标实例
qlib.init(provider_uri='/data/data_liy/qlib/etf_data')

# 创建指标实例
indicator = PriceStartupIndicator()

# print("ETF价格突然启动检测指标包含以下5个子信号:")
# print("1. 成交量放大信号 - 检测成交量是否显著超过近期平均")
# print("2. 价格动量信号 - 检测价格是否快速上涨") 
# print("3. 阻力位突破信号 - 检测是否突破前期高点")
# print("4. 波动率扩张信号 - 检测市场波动是否放大")
# print("5. 价格趋势信号 - 检测价格是否呈现上升趋势")
# print("\n综合信号: 当至少3个子信号同时出现时，判定为价格突然启动")


[12155:MainThread](2025-08-05 16:27:10,216) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[12155:MainThread](2025-08-05 16:27:10,557) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[12155:MainThread](2025-08-05 16:27:10,558) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/data_liy/qlib/etf_data')}


In [3]:
# 使用示例：分析创业板ETF的启动信号
instruments = ['159302.SZ']  # 创业板指数ETF
start_time = '2024-01-01'
end_time = '2024-12-31'

print(f"分析标的: {instruments[0]} (创业板指数ETF)")
print(f"时间范围: {start_time} 到 {end_time}")

# 获取启动信号数据
data = indicator.analyze_etf_startup(instruments, start_time, end_time)
print(f"\n数据形状: {data.shape}")

# 显示最近的启动信号
startup_signals = data[data['startup_signal'] == 1]
print(f"\n检测到 {len(startup_signals)} 次启动信号")

if len(startup_signals) > 0:
    print("\n最近5次启动信号:")
    recent_signals = startup_signals.tail()
    for idx, row in recent_signals.iterrows():
        date = idx[1].strftime('%Y-%m-%d')
        strength = row['startup_strength']
        close_price = row['close']
        print(f"  {date}: 强度={strength:.0f}, 收盘价={close_price:.2f}")
else:
    print("该时间段内未检测到启动信号")


分析标的: 159302.SZ (创业板指数ETF)
时间范围: 2024-01-01 到 2024-12-31

数据形状: (80, 12)

检测到 0 次启动信号
该时间段内未检测到启动信号


In [5]:

# 直接使用qlib表达式查询启动信号
print("=== 直接使用qlib表达式查询 ===")

# 获取可用的字段表达式
fields = create_startup_fields()

# 构建查询字段
query_fields = [
    fields['close'],           # 收盘价
    fields['volume'],          # 成交量
    fields['startup_signal'],  # 启动信号
    fields['startup_strength'] # 启动强度
]

print("查询字段表达式:")
for i, field in enumerate(query_fields):
    print(f"{i+1}. {field}")
#liy --最关键的代码是这里
# 执行查询
result = D.features(
    instruments=['560660.SH'],
    fields=query_fields,
    start_time='2015-11-01',
    end_time='2025-12-31'
)



# 重命名列便于理解
result.columns = ['收盘价', '成交量', '启动信号', '启动强度']
result

# print(f"\n查询结果(最近10天):")
# print(result.tail(10))

# 显示最近启动信号
recent_signals = result[result['启动强度'] >= 1]
if len(recent_signals) > 0:
    print(f"\n最近启动信号({len(recent_signals)}次):")
    print(recent_signals.tail())
else:
    print("\n最近期间无启动信号")


=== 直接使用qlib表达式查询 ===
查询字段表达式:
1. $close
2. $volume
3. (($volume / Mean($volume, 20) > 1.05) + (($close / Ref($close, 1) - 1) > 0.01) + ($close > Max($high, 20) * (1 + 0.002)) + (Std($close, 10) > Mean(Std($close, 10), 20) * 1.03) + ($close > Ref($close, 1))) >= 3
4. (($volume / Mean($volume, 20) > 1.05) + (($close / Ref($close, 1) - 1) > 0.01) + ($close > Max($high, 20) * (1 + 0.002)) + (Std($close, 10) > Mean(Std($close, 10), 20) * 1.03) + ($close > Ref($close, 1)))

最近启动信号(703次):
                         收盘价       成交量  启动信号  启动强度
instrument datetime                               
560660.SH  2025-07-11  1.216   26448.0   0.0   1.0
           2025-07-15  1.310  175065.0   0.0   1.0
           2025-07-16  1.307  160895.0   0.0   1.0
           2025-07-17  1.344   90425.0   0.0   1.0
           2025-07-18  1.344   99642.0   0.0   1.0


In [15]:
result

收盘价           成交量  启动信号  启动强度
instrument datetime                                   
159915.SZ  2015-11-02  2.311  2.367417e+06   0.0   0.0
           2015-11-03  2.305  2.026363e+06   0.0   0.0
           2015-11-04  2.489  3.304478e+06   0.0   1.0
           2015-11-05  2.443  4.424902e+06   0.0   0.0
           2015-11-06  2.548  3.280866e+06   0.0   1.0
...                      ...           ...   ...   ...
           2025-07-14  2.177  6.661325e+06   0.0   0.0
           2025-07-15  2.212  1.427318e+07   0.0   1.0
           2025-07-16  2.206  8.014570e+06   0.0   0.0
           2025-07-17  2.247  9.615694e+06   0.0   1.0
           2025-07-18  2.252  9.440410e+06   0.0   1.0

[2362 rows x 4 columns]